# stata

> Wrapper for `pystata`/`sfi` (which require a Stata installation)

`sfi` is [Stata's python API](https://www.stata.com/python/api17/index.html), originally intended for interacting with Stata from python *within Stata*. As such, it can only be imported with Stata running.
`pystata.stata.run` [enables running Stata code from python](https://www.stata.com/python/pystata/stata.html#pystata.stata.run).

In [ ]:
#| default_exp stata
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.config import launch_stata
from nbstata.utils import HiddenPrints

In [ ]:
from textwrap import dedent

## Simple wrappers

For one thing, these remove the need to constantly import `pystata` and/or `sfi` within functions to avoid running afoul of CI tests in an environment without Stata installed.

In [ ]:
#| export
def run_direct(*args, **kwargs):
    import pystata
    return pystata.stata.run(*args, **kwargs)

[https://www.stata.com/python/api16/Macro.html#sfi.Macro.getLocal](https://www.stata.com/python/api16/Macro.html#sfi.Macro.getLocal)

In [ ]:
#| export
def get_local(name):
    import sfi
    return sfi.Macro.getLocal(name)

https://www.stata.com/python/api16/Macro.html#sfi.Macro.getGlobal

In [ ]:
#| export
def get_global(name):
    import sfi
    return sfi.Macro.getGlobal(name)

https://www.stata.com/python/api16/SFIToolkit.html#sfi.SFIToolkit.formatValue

In [ ]:
#| export
def stata_formatted(value, s_format):
    import sfi
    return sfi.SFIToolkit.formatValue(value, s_format)

`variable_names` uses 'getVarCount' and 'getVarName': https://www.stata.com/python/api16/Data.html 

In [ ]:
#| export
def variable_names():
    from sfi import Data
    return [Data.getVarName(i) for i in range(Data.getVarCount())]

In [ ]:
#| eval: false
launch_stata(splash=False)
run_direct("gen var1 = 1", quietly=True)
variable_names()

['var1']

## Simple utilities

In [ ]:
#| export
def obs_count():
    """Count the number of observations"""
    import sfi
    return sfi.Data.getObsTotal()

In [ ]:
#| eval: false
obs_count()

0

The following function is not currently used and is likely made unnecessary by `sfi.SFIToolkit.macroExpand`.

In [ ]:
#| export
def resolve_macro(macro):
    macro = macro.strip()
    if macro.startswith("`") and macro.endswith("'"):
        macro = get_local(macro[1:-1])
    elif macro.startswith("$_"):
        macro = get_local(macro[2:])
    elif macro.startswith("${") and macro.endswith("}"):
        macro = get_global(macro[2:-1])
    elif macro.startswith("$"):
        macro = get_global(macro[1:])
    return macro

In [ ]:
#| eval: false
run_direct('global test = "hello world"')
resolve_macro("$test")

'hello world'

In [ ]:
#| hide
#| eval: false
resolve_macro("${test}")

'hello world'

## Run multi-line Stata commands no-echo

`pystata.stata.run` can only suppress the "echo" of single commands, not multi-line Stata code:

In [ ]:
#| eval: false
run_direct('disp "test 1"', echo=False)

test 1


In [ ]:
#| eval: false
two_lines_of_code = dedent('''\
    disp "test 1"
    disp "test 2"
    ''')
run_direct(two_lines_of_code, echo=False)


. disp "test 1"
test 1

. disp "test 2"
test 2

. 


As a workaround when echo is not desired, we can run multiple commands as a Stata program:

In [ ]:
#| eval: false
run_direct(f"program temp_nbstata_program_name\n{two_lines_of_code}\nend\n", quietly=True)

In [ ]:
#| eval: false
run_direct("temp_nbstata_program_name", quietly=False, inline=True, echo=False)

test 1
test 2


In [ ]:
#| eval: false
run_direct(f"program drop temp_nbstata_program_name", quietly=True)

The first `run` (defining the temp. program) prints an unwanted blank line despite setting "quietly" to True, so we block its printed output in the function version below.

(Note: This and the following two functions assume input Stata code standardized by `standardize_code`, which will be ensured by the `break_out_prog_blocks` within the ultimate `dispatch_run` wrapper function.)

In [ ]:
#| export
def run_as_program(std_non_prog_code, prog_def_option_code=""):
    _program_name = "temp_nbstata_program_name"
    _program_define_code = (
        f"program {_program_name}"
        f"{', ' if prog_def_option_code else ''}{prog_def_option_code}\n"
        f"{std_non_prog_code}\n"
        "end\n"
    )
    try:
        with HiddenPrints():
            run_direct(_program_define_code, quietly=True)
        run_direct(_program_name, quietly=False, inline=True, echo=False)
    finally:
        run_direct(f"program drop {_program_name}", quietly=True)

In [ ]:
#| eval: false
run_as_program(two_lines_of_code)

test 1
test 2


In [ ]:
#| hide
#| eval: false
run_as_program(two_lines_of_code, "sclass")

test 1
test 2


Not all code can be run within a program without modification, however: 
1. Programs cannot be defined within another program, nor can python or mata blocks be run.
2. A program definition is a different scope for locals, so:
    * the program code does not have access to locals defined previously, and
    * locals set within the program code do not persist outside of it.

In [ ]:
from fastcore.test import ExceptionExpected

In [ ]:
#| eval: false
with ExceptionExpected(SystemError):
    run_as_program("""\
        program define prog1
            disp 1
        end
        """)

In [ ]:
#| eval: false
run_direct("local test1 = 1")
run_direct("disp `test1'")

1


In [ ]:
#| eval: false
run_as_program("""\
    disp `test1'
    local test2 = 2""")

In [ ]:
#| eval: false
run_direct("""\
    disp `test1'
    disp `test2' """)


.     disp `test1'
1

.     disp `test2' 


. 


In [ ]:
#| export
def run_non_prog_noecho(std_non_prog_code, run_as_prog=run_as_program):
    if len(std_non_prog_code.splitlines()) <= 1:  # to keep it simple when we can
        run_direct(std_non_prog_code, quietly=False, inline=True, echo=False)
    else:
        run_as_prog(std_non_prog_code)

In [ ]:
#| eval: false
run_non_prog_noecho('disp "test 1"')

test 1


In [ ]:
#| eval: false
run_non_prog_noecho(two_lines_of_code)

test 1
test 2


In [ ]:
#| export
def run_prog_noecho(std_prog_code):
    if std_prog_code.splitlines()[0] in {'mata', 'mata:'}:  # b/c 'quietly' blocks mata output
        run_direct(std_prog_code, quietly=False, inline=True, echo=False)
    else:
        run_direct(std_prog_code, quietly=True, inline=True, echo=False)

Other programs (that is, Stata's `program define`, as well as [mata](https://www.stata.com/manuals/m-1first.pdf) or [python](https://www.stata.com/stata-news/news35-3/python-blogs/) blocks) cannot be defined/run within a Stata program, however. Instead, we will just run them directly, quietly to prevent echo, except for the case of mata programs, in which case `quietly` would block the output.

In [ ]:
#| eval: false
#| hide
run_as_program("capture program drop display1")

In [ ]:
#| eval: false
prog_block_code = dedent("""\
    program define display1
        disp "display1 output"
    end
    """)
run_prog_noecho(prog_block_code)
run_direct("display1", echo=False)


display1 output


In [ ]:
#| eval: false
python_block_code = dedent("""\
    python:
    print("hello")
    end
    """)
run_prog_noecho(python_block_code)

hello



In [ ]:
#| eval: false
mata_block_code = dedent("""\
    mata:
    display("hello")
    end
    """)
run_prog_noecho(mata_block_code)


. mata:
------------------------------------------------- mata (type end to exit) -----
: display("hello")
hello

: end
-------------------------------------------------------------------------------

. 


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()